In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from dvc import api
from io import StringIO
from joblib import dump


# Cargar los datos y realizar el preprocesamiento
# df_ml = pd.read_csv('../Data/df_ML_final_2.csv')
df_path = api.read('../dataset/data_prueba_limpia.csv', remote='dataset-track')
df_ml = pd.read_csv(StringIO(df_path))


# Eliminar las columnas 'target'  del DataFrame original y guardarlo en otro Df
df_target = df_ml[['target']]
df_ml = df_ml.drop(columns=['target'])

# PCA variables relacionadas
variables = ['decil_ingreso_ENIGH', 'liquidez_porcentual', 'costo_de_vida']
pca = PCA(n_components=1)
df_ml['ENIGH'] = pca.fit_transform(df_ml[variables])
df_ml = df_ml.drop(columns=variables)

# Clusterización 
optimal_clusters= 4
scaler = StandardScaler()
df_ml_scaled = scaler.fit_transform(df_ml)
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
kmeans.fit(df_ml_scaled)
df_ml['cluster'] = kmeans.labels_

# concatenar los dfs
df_ml = pd.concat([df_ml, df_target], axis=1)

# Score y cluster
X = df_ml.drop(['target', 'cluster'], axis=1)
y_score = df_ml['target'] 
y_cluster = df_ml['cluster'] 

X_train, X_test, y_train_score, y_test_score = train_test_split(X, y_score, test_size=0.3, 
                                                                random_state=42)
y_train_cluster, y_test_cluster = train_test_split(y_cluster, test_size=0.3, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo de regresión Random Forest para predecir score crediticio
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_scaled, y_train_score)

# Modelo de clasificación Random Forest para predecir el cluster
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_scaled, y_train_cluster)

dump(rf_reg, 'model/model_reg.pkl')
dump(rf_clf, 'model/model_clf.pkl')
dump(scaler, 'model/model_scaler.pkl')
# # Función para clasificar a los clientes
# def clasificar_clientes(ingreso, 
#                         antiguedad_laboral_meses, 
#                         tiempo_desempleado,
#                         trabajos_ultimos_5, 
#                         semanasCotizadas, 
#                         edad, 
#                         crecimiento_ingreso, 
#                         crecimiento_gral, 
#                         ENIGH):

#     # Crear el DataFrame para el cliente
#     cliente = pd.DataFrame(data={
#         'ingreso': [ingreso],
#         'antiguedad_laboral_meses': [antiguedad_laboral_meses],
#         'tiempo_desempleado': [tiempo_desempleado],
#         'trabajos_ultimos_5': [trabajos_ultimos_5],
#         'semanasCotizadas': [semanasCotizadas],
#         'edad': [edad],
#         'crecimiento_ingreso': [crecimiento_ingreso],
#         'crecimiento_gral' : [crecimiento_gral],
#         'ENIGH' : [ENIGH]
#     }, index=[0])

#     # Escalar los datos del cliente
#     cliente_scaled = scaler.transform(cliente)

#     # Obtener la predicción del score (modelo de regresión) y del cluster (modelo de clasificación) 
#     # para el cliente
#     puntuacion_credito = rf_reg.predict(cliente_scaled)[0]
#     cluster_predicho = rf_clf.predict(cliente_scaled)[0]

#     return puntuacion_credito, cluster_predicho


In [9]:
# score_cliente, cluster_cliente = clasificar_clientes(ingreso=45000,
#                                                         antiguedad_laboral_meses=50, 
#                                                         tiempo_desempleado=0,
#                                                         trabajos_ultimos_5=1, 
#                                                         semanasCotizadas=1000, 
#                                                         edad=32, 
#                                                         crecimiento_ingreso=265.38, 
#                                                         crecimiento_gral= 0, 
#                                                         ENIGH=9)

# display(("El score del cliente es <b>{}</b> y pertenece al cluster <b>{}</b>".format(round(score_cliente*1000, 2), cluster_cliente)))

'El score del cliente es <b>678.17</b> y pertenece al cluster <b>2</b>'

In [10]:
# df_ml.iloc[4589]

In [11]:
# score_cliente2, cluster_cliente2 = clasificar_clientes(ingreso=0,
#                                                         antiguedad_laboral_meses=0, 
#                                                         tiempo_desempleado=13,
#                                                         trabajos_ultimos_5=2, 
#                                                         semanasCotizadas=198, 
#                                                         edad=29, 
#                                                         crecimiento_ingreso=0, 
#                                                         crecimiento_gral=2, 
#                                                         ENIGH=-105)

# display(("El score del cliente es <b>{}</b> y pertenece al cluster <b>{}</b>".format(round(score_cliente2*1000, 2), cluster_cliente2)))

'El score del cliente es <b>433.33</b> y pertenece al cluster <b>0</b>'

In [12]:
# df_ml.iloc[15232]

In [13]:
# score_cliente3, cluster_cliente3 = clasificar_clientes(ingreso=13472,
#                                                         antiguedad_laboral_meses=62, 
#                                                         tiempo_desempleado=0,
#                                                         trabajos_ultimos_5=1, 
#                                                         semanasCotizadas=561, 
#                                                         edad=46, 
#                                                         crecimiento_ingreso=0, 
#                                                         crecimiento_gral=1.481252, 
#                                                         ENIGH=19.892769)

# display(("El score del cliente es <b>{}</b> y pertenece al cluster <b>{}</b>".format(round(score_cliente3*1000, 2), cluster_cliente3)))

'El score del cliente es <b>652.5</b> y pertenece al cluster <b>2</b>'